In [2]:
list_1 = [1,4,6,2,7,4,9,3,3,4]

In [3]:
list_1.sort()
print(list_1[4::-1])

[4, 3, 3, 2, 1]


In [4]:
import pandas as pd

df = pd.DataFrame({'ChildId':['id1', 'id2', 'id3', 'id4', 'id5'],
                  'Age first contact':[6,12,11,1,19],
                  'Gender':['M','m', 'F', '', 'F' ],
                  'Birthday':['01/01/2002', '02/02/2003', pd.NA, '03/03/2023', '06/01/2012'],
                  'CP Plan?':['Y', 'n', 'N', 'No', 'yES'],})


In [5]:
print(df.head(5))

  ChildId  Age first contact Gender    Birthday CP Plan?
0     id1                  6      M  01/01/2002        Y
1     id2                 12      m  02/02/2003        n
2     id3                 11      F        <NA>        N
3     id4                  1         03/03/2023       No
4     id5                 19      F  06/01/2012      yES


# Working with dates

In [6]:
df['Birthday'] = pd.to_datetime(df['Birthday'], format='%d/%m/%Y', errors='coerce')

df['Age at 31 March'] = pd.to_datetime('31/03/2024', dayfirst=True) - df['Birthday']
df['Age at 31 March'] = df['Age at 31 March'] / pd.Timedelta('365 days')
df['Age at 31 March'] = df['Age at 31 March'].round().astype('int', errors='ignore')

print(df)

  ChildId  Age first contact Gender   Birthday CP Plan?  Age at 31 March
0     id1                  6      M 2002-01-01        Y             22.0
1     id2                 12      m 2003-02-02        n             21.0
2     id3                 11      F        NaT        N              NaN
3     id4                  1        2023-03-03       No              1.0
4     id5                 19      F 2012-01-06      yES             12.0


In [7]:
df['Age now'] = pd.to_datetime('today').normalize() - df['Birthday']
df['Age now'] = df['Age now'] / pd.Timedelta('365 days')
df['Age now'] = df['Age now'].round().astype('int', errors='ignore')

print(df)
print(df.dtypes)

  ChildId  Age first contact Gender   Birthday CP Plan?  Age at 31 March  \
0     id1                  6      M 2002-01-01        Y             22.0   
1     id2                 12      m 2003-02-02        n             21.0   
2     id3                 11      F        NaT        N              NaN   
3     id4                  1        2023-03-03       No              1.0   
4     id5                 19      F 2012-01-06      yES             12.0   

   Age now  
0     22.0  
1     21.0  
2      NaN  
3      1.0  
4     12.0  
ChildId                      object
Age first contact             int64
Gender                       object
Birthday             datetime64[ns]
CP Plan?                     object
Age at 31 March             float64
Age now                     float64
dtype: object


# Gender - mapping and regex replacing

In [13]:
print(df['Gender'].unique())

#gender_dict = {"M":"m", "m":"m", "F":"f", "f":"f"}
#df['Gender'] = df['Gender'].map(gender_dict)

df['Gender']=df['Gender'].str.upper()

df['Gender']=df['Gender'].replace(r'^\s*$', pd.NA, regex=True)

df['Gender']
df

['M' 'F' <NA> 'm']


,ChildId,Age first contact,Gender,Birthday,CP Plan?,Age at 31 March,Age now,Current Age,Age at End of Reporting Period,NHS Number
0,id1,6,M,2002-01-01,y,22.0,22.0,NaN,NaN,303
1,id2,12,M,2003-02-02,n,21.0,21.0,NaN,NaN,3u5029
2,id3,11,F,NaT,n,NaN,NaN,NaN,NaN,gqw3
3,id4,1,<NA>,2023-03-03,n,1.0,1.0,NaN,NaN,avsgvb
4,id5,19,F,2012-01-06,y,12.0,12.0,NaN,NaN,varwvw
5,id6,9,M,2014-02-28,NaN,NaN,NaN,10.0,10.0,78132


# Cleaning the CP plan column - lambda functions

In [9]:
cp_cleaner = lambda row: 'y' if 'y' in row else 'n' if 'n' in row else pd.NA

df['CP Plan?'] = df['CP Plan?'].str.lower().apply(cp_cleaner)

print(df['CP Plan?'])

0    y
1    n
2    n
3    n
4    y
Name: CP Plan?, dtype: object


Will is deliberately not teaching loops until next week because people overuse them
 - there is usually a better way
(ideally, wouldn't teach them at all! but they can be useful)

# Appending row to DataFrame

In [10]:
new_child = {
   'ChildId':['id6'],
   'Current Age': [10],
   'Age first contact':[9],
   'Gender':['m'],
   'Birthday':[pd.to_datetime('28/02/2014', dayfirst=True)],
   'Age at End of Reporting Period':[10],
   'NHS Number':['78132']
}

new_child = pd.DataFrame(new_child)
df = pd.concat([df, new_child], ignore_index=True)

df.drop_duplicates(subset='ChildId', keep='last', inplace=True)

df


,ChildId,Age first contact,Gender,Birthday,CP Plan?,Age at 31 March,Age now,Current Age,Age at End of Reporting Period,NHS Number
0,id1,6,M,2002-01-01,y,22.0,22.0,NaN,NaN,NaN
1,id2,12,M,2003-02-02,n,21.0,21.0,NaN,NaN,NaN
2,id3,11,F,NaT,n,NaN,NaN,NaN,NaN,NaN
3,id4,1,<NA>,2023-03-03,n,1.0,1.0,NaN,NaN,NaN
4,id5,19,F,2012-01-06,y,12.0,12.0,NaN,NaN,NaN
5,id6,9,m,2014-02-28,NaN,NaN,NaN,10.0,10.0,78132


# Merge two dataframes

In [11]:
nhs_numbers = pd.DataFrame([
                           {'ChildId':'id1',
                           'NHS Number': '303',},
                           {'ChildId':'id2',
                           'NHS Number': '3u5029',},
                           {'ChildId':'id3',
                           'NHS Number': 'gqw3',},
                           {'ChildId':'id4',
                           'NHS Number': 'avsgvb',},
                           {'ChildId':'id5',
                           'NHS Number': 'varwvw',},
                           ])

df = pd.merge(df, nhs_numbers, how='left', left_on='ChildId', right_on='ChildId', suffixes=['_original','_nhs_num'])

df

,ChildId,Age first contact,Gender,Birthday,CP Plan?,Age at 31 March,Age now,Current Age,Age at End of Reporting Period,NHS Number_original,NHS Number_nhs_num
0,id1,6,M,2002-01-01,y,22.0,22.0,NaN,NaN,NaN,303
1,id2,12,M,2003-02-02,n,21.0,21.0,NaN,NaN,NaN,3u5029
2,id3,11,F,NaT,n,NaN,NaN,NaN,NaN,NaN,gqw3
3,id4,1,<NA>,2023-03-03,n,1.0,1.0,NaN,NaN,NaN,avsgvb
4,id5,19,F,2012-01-06,y,12.0,12.0,NaN,NaN,NaN,varwvw
5,id6,9,m,2014-02-28,NaN,NaN,NaN,10.0,10.0,78132,NaN


# Rename a column

In [12]:
df['NHS Number_original'] = df['NHS Number_original'].fillna(df['NHS Number_nhs_num'])
df.drop('NHS Number_nhs_num', inplace=True, axis=1)
df.rename({'NHS Number_original':'NHS Number'}, axis=1, inplace=True)

df

,ChildId,Age first contact,Gender,Birthday,CP Plan?,Age at 31 March,Age now,Current Age,Age at End of Reporting Period,NHS Number
0,id1,6,M,2002-01-01,y,22.0,22.0,NaN,NaN,303
1,id2,12,M,2003-02-02,n,21.0,21.0,NaN,NaN,3u5029
2,id3,11,F,NaT,n,NaN,NaN,NaN,NaN,gqw3
3,id4,1,<NA>,2023-03-03,n,1.0,1.0,NaN,NaN,avsgvb
4,id5,19,F,2012-01-06,y,12.0,12.0,NaN,NaN,varwvw
5,id6,9,m,2014-02-28,NaN,NaN,NaN,10.0,10.0,78132
